<a href="https://colab.research.google.com/github/tiwari-arpit/nlp/blob/main/SeNER_Hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [2]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoConfig, AutoModelForTokenClassification, AutoTokenizer

In [7]:
model_name = "arpit-tiwari/distilbert-finetuned-hindi-ner"
model_checkpoint = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/810 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset
lang = 'hi'
data = load_dataset('ai4bharat/naamapadam',lang)

README.md:   0%|          | 0.00/8.65k [00:00<?, ?B/s]

naamapadam.py:   0%|          | 0.00/2.86k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/93.3M [00:00<?, ?B/s]

0001.parquet:   0%|          | 0.00/5.48M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/93.5k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/985787 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/867 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13460 [00:00<?, ? examples/s]

In [4]:
hindi_data = data
hindi_data['train'] = data['train'].select(range(10000,50000))

In [5]:
tags = hindi_data['train'].features['ner_tags'].feature
def create_tag_name(batch):
  tag_name = {'ner_tags_str': [ tags.int2str(idx) for idx in batch['ner_tags']]}
  return tag_name

hindi_data = hindi_data.map(create_tag_name)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/867 [00:00<?, ? examples/s]

Map:   0%|          | 0/13460 [00:00<?, ? examples/s]

In [8]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = hindi_data.map(tokenize_and_align_labels,batched=True,remove_columns=hindi_data['train'].column_names)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/867 [00:00<?, ? examples/s]

Map:   0%|          | 0/13460 [00:00<?, ? examples/s]

In [9]:
class ArrowAttention(nn.Module):
    def __init__(self, hidden_size, window_size, num_heads=8):
        super().__init__()
        self.window_size = window_size
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads
        self.cls_attention = nn.MultiheadAttention(hidden_size, num_heads)
        self.local_attention = nn.MultiheadAttention(hidden_size, num_heads)

    def forward(self, hidden_states):
        batch_size, seq_len, hidden_size = hidden_states.size()

        cls_token = hidden_states[:, 0:1, :]
        cls_token = cls_token.transpose(0, 1)
        global_output, _ = self.cls_attention(
            cls_token,
            hidden_states.transpose(0, 1),
            hidden_states.transpose(0, 1)
        )
        global_output = global_output.transpose(0, 1)

        if seq_len > 1:
            local_outputs = []
            for i in range(1, seq_len):
                start = max(1, i - self.window_size)
                end = min(seq_len, i + self.window_size + 1)

                window = hidden_states[:, start:end, :]
                current_token = hidden_states[:, i:i+1, :]

                current_token = current_token.transpose(0, 1)
                window = window.transpose(0, 1)

                local_output, _ = self.local_attention(
                    current_token,
                    window,
                    window
                )
                local_output = local_output.transpose(0, 1)
                local_outputs.append(local_output)

            if local_outputs:
                local_output = torch.cat(local_outputs, dim=1)
                output = torch.cat([global_output, local_output], dim=1)
            else:
                output = global_output
        else:
            output = global_output

        return output

In [10]:
class LogNScaling(nn.Module):
    def __init__(self, hidden_size):
        super(LogNScaling, self).__init__()
        self.scale = nn.Parameter(torch.tensor(1.0 / (hidden_size ** 0.5)))

    def forward(self, attention_scores):
        return attention_scores * self.scale

In [11]:
class BiSPA(nn.Module):
    def __init__(self, hidden_size, window_size, num_heads=8):
        super(BiSPA, self).__init__()
        self.window_size = window_size
        self.num_heads = num_heads
        self.horizontal_attention = nn.MultiheadAttention(hidden_size, num_heads)
        self.vertical_attention = nn.MultiheadAttention(hidden_size, num_heads)
        self.mlp = nn.Sequential(
            nn.Linear(2 * hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size)
        )

    def forward(self, hidden_states):
        batch_size, seq_len, hidden_size = hidden_states.size()
        hidden_states_t = hidden_states.transpose(0, 1)

        # Horizontal attention
        horizontal_outputs = []
        for i in range(seq_len):
            start = max(0, i - self.window_size)
            end = min(seq_len, i + self.window_size + 1)
            horizontal_input = hidden_states_t[start:end, :, :]
            query = hidden_states_t[i:i+1, :, :]

            horizontal_output, _ = self.horizontal_attention(
                query,
                horizontal_input,
                horizontal_input
            )
            horizontal_outputs.append(horizontal_output)

        horizontal_output = torch.cat(horizontal_outputs, dim=0)

        # Vertical attention
        vertical_outputs = []
        for i in range(seq_len):
            query = hidden_states_t[i:i+1, :, :]
            vertical_output, _ = self.vertical_attention(
                query,
                hidden_states_t,
                hidden_states_t
            )
            vertical_outputs.append(vertical_output)

        vertical_output = torch.cat(vertical_outputs, dim=0)

        # Combine outputs
        combined_output = torch.cat([horizontal_output, vertical_output], dim=-1)
        output = self.mlp(combined_output)
        output = output.transpose(0, 1)

        return output

In [12]:
class SeNER(nn.Module):
    def __init__(self, model_name, num_labels, window_size=128):
        super(SeNER, self).__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.base_model = AutoModel.from_pretrained(model_name, config=self.config)
        self.num_labels = num_labels
        self.arrow_attention = ArrowAttention(self.config.hidden_size, window_size)
        self.log_n_scaling = LogNScaling(self.config.hidden_size)
        self.bispa = BiSPA(self.config.hidden_size, window_size)
        self.classifier = nn.Linear(self.config.hidden_size, num_labels)
        self.dropout = nn.Dropout(self.config.hidden_size_dropout if hasattr(self.config, 'hidden_size_dropout') else 0.1)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state

        attended_output = self.arrow_attention(sequence_output)

        if attended_output.size(1) > 0:
            cls_token = attended_output[:, 0:1, :]
            scaled_cls = self.log_n_scaling(cls_token)

            if attended_output.size(1) > 1:
                attended_output = torch.cat([scaled_cls, attended_output[:, 1:, :]], dim=1)
            else:
                attended_output = scaled_cls

        sequence_output = self.bispa(attended_output)
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            active_loss = labels.view(-1) != -100
            active_logits = logits.view(-1, self.num_labels)[active_loss]
            active_labels = labels.view(-1)[active_loss]
            loss = loss_fct(active_logits, active_labels)

        return {'loss': loss, 'logits': logits} if loss is not None else {'logits': logits}


In [22]:
from transformers import DataCollatorForTokenClassification
data_colator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [13]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [15]:
import zipfile
import os

# if not os.path.exists(extract_dir):
#     os.makedirs(extract_dir)

with zipfile.ZipFile("/content/MyDrive/MyDrive/SeNER.zip", 'r') as zip_ref:
    zip_ref.extractall("extract_dir")

os.listdir("/content/extract_dir/")

['content']

In [16]:
os.listdir("/content/extract_dir/content/SeNER")

['tokenizer_config.json',
 'model.safetensors',
 'vocab.txt',
 'training_args.bin',
 'special_tokens_map.json',
 'tokenizer.json']

In [17]:
num_labels = len(tags.names)
model = SeNER(model_name, num_labels)

In [19]:
from safetensors.torch import load_file
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

state_dict = load_file("/content/extract_dir/content/SeNER/model.safetensors")
model.load_state_dict(state_dict)
model.to(device)

SeNER(
  (base_model): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_

In [20]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./SeNER_Hindi",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    remove_unused_columns=False,
)

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].select(range(10000,40000)),
    eval_dataset=tokenized_dataset["test"].select(range(800)),
    data_collator = data_colator,
)

trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: arpit-tiwari (eshaan-rithesh2023-vit-chennai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.255300,0.260369


TrainOutput(global_step=7500, training_loss=0.2948045257568359, metrics={'train_runtime': 4814.76, 'train_samples_per_second': 6.231, 'train_steps_per_second': 1.558, 'total_flos': 0.0, 'train_loss': 0.2948045257568359, 'epoch': 1.0})

In [29]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.260369211435318, 'eval_runtime': 33.8008, 'eval_samples_per_second': 23.668, 'eval_steps_per_second': 5.917, 'epoch': 1.0}


In [30]:
trainer.save_model("./SeNER")

In [31]:
def predict_entities(model, tokenizer, text, id2label=None):
    inputs = tokenizer(text, return_offsets_mapping=True, return_tensors="pt", truncation=True, padding=True)
    offset_mapping = inputs.pop("offset_mapping")
    inputs = {k: v.to(next(model.parameters()).device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs["logits"]
    predictions = torch.argmax(logits, dim=-1)[0].cpu().numpy()
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    offset_mapping = offset_mapping[0].cpu().numpy()

    word_entities = []
    previous_word = None
    current_word = ""
    current_label = None

    for token, pred, offset in zip(tokens, predictions, offset_mapping):
        if offset[0] == 0 and offset[1] == 0:
            continue

        start, end = offset
        word = text[start:end]

        if token.startswith("##"):
            current_word += word
        else:
            if current_word:
                label_name = id2label[current_label] if id2label else current_label
                word_entities.append((current_word, label_name))
            current_word = word
            current_label = pred

    if current_word:
        label_name = id2label[current_label] if id2label else current_label
        word_entities.append((current_word, label_name))

    return word_entities


In [32]:
text = "इटली की पीएम जॉर्जिया मेलोनी ने पूरी दुनिया के लेफ्टिस्ट लीडर्स को पाखंडी बताया है। उन्होंने कहा कि दुनियाभर में मोदी, ट्रम्प और मेरे जैसे दक्षिणपंथी नेताओं के उभरने से सारे लेफ्टिस्ट नेता परेशान हो गए हैं|"
predictions = predict_entities(model, tokenizer, text)

print("Input text:", text)
print("Predictions:")
for token, pred in predictions:
    print(f"{token}: {tags.int2str(int(pred))}")


Input text: इटली की पीएम जॉर्जिया मेलोनी ने पूरी दुनिया के लेफ्टिस्ट लीडर्स को पाखंडी बताया है। उन्होंने कहा कि दुनियाभर में मोदी, ट्रम्प और मेरे जैसे दक्षिणपंथी नेताओं के उभरने से सारे लेफ्टिस्ट नेता परेशान हो गए हैं|
Predictions:
इटली: O
की: O
पीएम: O
जॉर्जिया: B-PER
मेलोनी: I-PER
ने: O
पूरी: O
दुनिया: O
के: O
लेफ्टिस्ट: O
लीडर्स: O
को: O
पाखंडी: O
बताया: O
है: O
।: O
उन्होंने: O
कहा: O
कि: O
दुनियाभर: O
में: O
मोदी: B-PER
,: O
ट्रम्प: O
और: O
मेरे: O
जैसे: O
दक्षिणपंथी: O
नेताओं: O
के: O
उभरने: O
से: O
सारे: O
लेफ्टिस्ट: O
नेता: O
परेशान: O
हो: O
गए: O
हैं: O
|: O


In [ ]:
tags

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)

In [25]:
torch.cuda.empty_cache()

In [26]:
import gc
gc.collect()

0

In [33]:
!zip -r SeNER.zip /content/SeNER

from google.colab import files
files.download('SeNER.zip')

  adding: content/SeNER/ (stored 0%)
  adding: content/SeNER/tokenizer_config.json (deflated 74%)
  adding: content/SeNER/model.safetensors (deflated 8%)
  adding: content/SeNER/vocab.txt (deflated 49%)
  adding: content/SeNER/training_args.bin (deflated 52%)
  adding: content/SeNER/special_tokens_map.json (deflated 80%)
  adding: content/SeNER/tokenizer.json (deflated 70%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>